In [ ]:
#implementing paper Efficient Detection of Knee Anterior Cruciate Ligament from Magnetic Resonance Imaging Using Deep Learning Approach 

In [5]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from collections import Counter
from torch.utils.data import Subset
from torch.utils.data import WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter
import numpy as np
import random

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L') 
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomVerticalFlip(p=0.5),   
    transforms.Resize((90, 90)),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

def undersample_class(dataset, majority_class, target_count):
    majority_indices = [i for i, (_, label) in enumerate(dataset) if label == majority_class]
    undersampled_majority_indices = np.random.choice(majority_indices, size=target_count, replace=False)
    return undersampled_majority_indices

class OversampleDataset(Dataset):
    def __init__(self, dataset, labels, minority_classes):
        self.dataset = dataset
        self.labels = labels
        self.minority_classes = minority_classes
        
        self.minority_indices = [i for i, label in enumerate(self.labels) if label in self.minority_classes]
        self.majority_indices = [i for i, label in enumerate(self.labels) if label not in self.minority_classes]
        
        self.oversampled_minority_indices = random.choices(self.minority_indices, k=min(len(self.majority_indices), len(self.minority_indices) * 5))
        
        self.indices = self.majority_indices + self.oversampled_minority_indices
    
    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        actual_idx = self.indices[idx] % len(self.dataset)
        return self.dataset[actual_idx]

def hybrid_sampling(dataset, labels, majority_class, minority_classes):
    class_counts = Counter(labels)
    
    target_count = 500
    undersampled_majority_indices = undersample_class(dataset, majority_class, target_count)
    
    minority_indices = [i for i, (_, label) in enumerate(dataset) if label in minority_classes]
    
    combined_indices = list(undersampled_majority_indices) + minority_indices
    
    return Subset(dataset, combined_indices)

data_dir = 'D:/project/new data'
dataset = CustomDataset(root_dir=data_dir, transform=transform)

labels = [label for _, label in dataset]  
class_counts = Counter(labels)

print("Class counts:", class_counts)

majority_class = max(class_counts, key=class_counts.get)
minority_classes = [cls for cls in class_counts if cls != majority_class]

print("Majority class:", majority_class)
print("Minority classes:", minority_classes)

dataset = hybrid_sampling(dataset, labels, majority_class, minority_classes)

curr_dataset = OversampleDataset(dataset, labels, minority_classes)

train_size = int(0.85 * len(curr_dataset))  
test_size = len(curr_dataset) - train_size
train_dataset, test_dataset = random_split(curr_dataset, [train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=32, 
                          shuffle=False)
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=32, 
                          shuffle=False)

print(f"Full dataset size: {len(curr_dataset)}")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout_prob=0.5):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(p=dropout_prob)  
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x) 
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out) 
        out = self.bn2(self.conv2(out))
        out += identity 
        out = F.relu(out)
        return out

class ResNet14Custom(nn.Module):
    def __init__(self, num_classes=3, dropout_prob=0.5):
        super(ResNet14Custom, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.dropout = nn.Dropout(p=dropout_prob) 
        
        self.layer2 = self._make_layer(16, 16, stride=1, num_blocks=2, dropout_prob=dropout_prob)
        
        self.layer3 = self._make_layer(16, 32, stride=2, num_blocks=2, dropout_prob=dropout_prob)
        
        self.layer4 = self._make_layer(32, 64, stride=2, num_blocks=2, dropout_prob=dropout_prob)
        
        self.avgpool = nn.AvgPool2d(4)
        
        self._to_linear = None
        self._get_conv_output_size()
        
        self.fc = nn.Linear(self._to_linear, num_classes)
    
    def _make_layer(self, in_channels, out_channels, stride, num_blocks, dropout_prob):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride, dropout_prob))  # First block may change dimensions
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels, dropout_prob=dropout_prob))  # Rest keep dimensions
        return nn.Sequential(*layers)
    
    def _get_conv_output_size(self):
        with torch.no_grad():
            
            x = torch.zeros(1, 1, 90, 90)
            
            x = F.relu(self.bn1(self.conv1(x)))
            
            x = self.layer2(x)
            
            x = self.layer3(x)
            
            x = self.layer4(x)
            
            x = self.avgpool(x)
            self._to_linear = x.numel()

    def forward(self, x):

        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout(x)  
        
        x = self.layer2(x)
        
        x = self.layer3(x)
        
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

model = ResNet14Custom(num_classes=3, dropout_prob=0)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5)


def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

num_epochs = 120
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    
    train_accuracy = calculate_accuracy(train_loader, model)
    test_accuracy = calculate_accuracy(test_loader, model)
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Checking directory: D:/project/new data\0
Loaded image: D:/project/new data\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/new data\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/new data\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/new data\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/new data\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/new data\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/new data\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/new data\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/new data\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/new data\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/new data\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/new data\0\406320_series9_slice2_R_roi.jpg
Loaded image: D:/project/new data\0\412857_series8_slice0_L_roi.jpg
Loaded image: D:/project/new data\0\412857_series8_slice1_L_roi.jpg
Loaded

In [ ]:
#training accuracy : 100%, test accuracy : 95.85%

In [ ]:
#implementing paper Improved Deep Convolutional Neural Network to Classify Osteoarthritis from Anterior Cruciate Ligament Tear Using Magnetic Resonance Imaging

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from collections import Counter
from torch.utils.data import Subset
from torch.utils.data import WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L') 
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

transform = transforms.Compose([ 
    transforms.Resize((90, 90)),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

data_dir = 'D:/project/with_roi_histactual3'
curr_dataset = CustomDataset(root_dir=data_dir, transform=transform)

train_size = int(0.85 * len(curr_dataset))  
test_size = len(curr_dataset) - train_size
train_dataset, test_dataset = random_split(curr_dataset, [train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=6, 
                          shuffle=False)
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=6, 
                          shuffle=False)

print(f"Full dataset size: {len(curr_dataset)}")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

class MRI_CNN(nn.Module):
    
    def __init__(self, num_classes=3):
        super(MRI_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self._to_linear = None
        self._get_conv_output_size()
        self.fc1 = nn.Linear(self._to_linear, 64) 
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)
        self._initialize_weights()
        
    def _get_conv_output_size(self):
        with torch.no_grad():
            
            x = torch.zeros(1, 1, 90, 90)
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            self._to_linear = x.numel()
    
    def _initialize_weights(self):

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                     
    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        return x

model = MRI_CNN(num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5)

def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    
    train_accuracy = calculate_accuracy(train_loader, model)
    test_accuracy = calculate_accuracy(test_loader, model)
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")


 

Checking directory: D:/project/with_roi_histactual3\0
Loaded image: D:/project/with_roi_histactual3\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual3\0\406320_series9_sli

In [ ]:
#training accuracy : 94.42%, test accuracy : 77.48%

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
from torch.utils.data import Subset
from torch.utils.data import WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Check for GPU availability and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
                
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Define transformations
transform = transforms.Compose([ 
    transforms.Resize((90, 90)),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

# Specify data directory
data_dir = 'D:/project/with_roi_histactual3'
curr_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training and testing
train_size = int(0.85 * len(curr_dataset))  
test_size = len(curr_dataset) - train_size
train_dataset, test_dataset = random_split(curr_dataset, [train_size, test_size])

# Create DataLoaders with pin_memory=True if using GPU for faster data transfer
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=6, 
                          shuffle=True,  # It's better to shuffle training data
                          pin_memory=True if torch.cuda.is_available() else False)
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=6, 
                         shuffle=False,
                         pin_memory=True if torch.cuda.is_available() else False)

print(f"Full dataset size: {len(curr_dataset)}")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

class MRI_CNN(nn.Module):
    def __init__(self, num_classes=3):
        super(MRI_CNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=5, padding=2)
        
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self._to_linear = None
        self._get_conv_output_size()
        
        # Fully connected layers
        self.fc1 = nn.Linear(self._to_linear, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, num_classes)
                
        # Dropout layer
        self.dropout = nn.Dropout(0.2)
        self._initialize_weights()
        
    def _get_conv_output_size(self):
        with torch.no_grad():
            x = torch.zeros(1, 1, 90, 90)
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = self.pool(F.relu(self.conv3(x)))
            self._to_linear = x.numel()
    
    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                      
    def forward(self, x):
        # Convolutional layers with ReLU and MaxPool
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        
        # Flatten the output from conv layers
        x = x.view(x.size(0), -1)
        
        # Fully connected layers with ReLU and Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)  # Removed ReLU for the final layer (typically combined with loss function)
        
        return x

# Instantiate the model and move it to the device
model = MRI_CNN(num_classes=3).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=5)

def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            # Move data to the device
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # Move data to the device
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # Calculate accuracy
    train_accuracy = calculate_accuracy(train_loader, model)
    test_accuracy = calculate_accuracy(test_loader, model)
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")


In [ ]:
#training accuracy : 74.96%, testing accuracy : 69.97%

In [ ]:
#implementing above 2 architechtures with hybrid sampling

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from torch.utils.data import Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L') 
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Define transformations and other dataset functions as before

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomVerticalFlip(p=0.5),   
    transforms.Resize((90, 90)),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

# Define the sampling and dataset creation functions as before
# [Undersample, oversample, and hybrid_sampling functions here]

data_dir = 'D:/project/with_roi_histactual3'
dataset = CustomDataset(root_dir=data_dir, transform=transform)

labels = [label for _, label in dataset]  
class_counts = Counter(labels)

print("Class counts:", class_counts)

majority_class = max(class_counts, key=class_counts.get)
minority_classes = [cls for cls in class_counts if cls != majority_class]

dataset = hybrid_sampling(dataset, labels, majority_class, minority_classes)
curr_dataset = OversampleDataset(dataset, labels, minority_classes)

train_size = int(0.85 * len(curr_dataset))  
test_size = len(curr_dataset) - train_size
train_dataset, test_dataset = random_split(curr_dataset, [train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=6, 
                          shuffle=False)
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=6, 
                          shuffle=False)

print(f"Full dataset size: {len(curr_dataset)}")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

class MRI_CNN(nn.Module):
    
    def __init__(self, num_classes=3):
        super(MRI_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self._to_linear = None
        self._get_conv_output_size()
        self.fc1 = nn.Linear(self._to_linear, 64) 
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)
        self._initialize_weights()
        
    def _get_conv_output_size(self):
        with torch.no_grad():
            x = torch.zeros(1, 1, 90, 90).to(device)
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            self._to_linear = x.numel()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                     
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

model = MRI_CNN(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5)

def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_accuracy = calculate_accuracy(train_loader, model)
    test_accuracy = calculate_accuracy(test_loader, model)
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")


Checking directory: D:/project/roi11\0
Loaded image: D:/project/roi11\0\329637-8_slice_14.png
Loaded image: D:/project/roi11\0\329637-8_slice_15.png
Loaded image: D:/project/roi11\0\329637-8_slice_16.png
Loaded image: D:/project/roi11\0\390116-9_slice_10.png
Loaded image: D:/project/roi11\0\390116-9_slice_11.png
Loaded image: D:/project/roi11\0\390116-9_slice_12.png
Loaded image: D:/project/roi11\0\390116-9_slice_13.png
Loaded image: D:/project/roi11\0\390116-9_slice_14.png
Loaded image: D:/project/roi11\0\390116-9_slice_15.png
Loaded image: D:/project/roi11\0\406320-9_slice_12.png
Loaded image: D:/project/roi11\0\406320-9_slice_13.png
Loaded image: D:/project/roi11\0\406320-9_slice_14.png
Loaded image: D:/project/roi11\0\412857-8_slice_14.png
Loaded image: D:/project/roi11\0\412857-8_slice_15.png
Loaded image: D:/project/roi11\0\412857-8_slice_16.png
Loaded image: D:/project/roi11\0\412857-8_slice_17.png
Loaded image: D:/project/roi11\0\415102-9_slice_11.png
Loaded image: D:/project/r

: 

In [ ]:
#Train accuracy : 100% , test accuracy : 96.57%